# ArUco Smoothing Visualization

Compare original vs smoothed ArUco poses to see the effect of interpolation on missing frames.

This notebook loads both the original and smoothed trajectory files and visualizes:
- 3D trajectories (original vs smoothed)
- Position differences over time
- Quaternion differences over time


In [38]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from scipy.spatial.transform import Rotation as R

# Load original and smoothed trajectories
ORIGINAL_PATH = Path("data/trajectory_20251215_163852_7.npz")
SMOOTHED_PATH = Path("data/trajectory_20251215_163852_7_smoothed.npz")

if not ORIGINAL_PATH.exists():
    raise FileNotFoundError(f"Original file not found: {ORIGINAL_PATH}")
if not SMOOTHED_PATH.exists():
    raise FileNotFoundError(f"Smoothed file not found: {SMOOTHED_PATH}. Run smooth_aruco_trajectory.py first.")

print(f"Loading original: {ORIGINAL_PATH}")
data_orig = np.load(ORIGINAL_PATH, allow_pickle=True)

print(f"Loading smoothed: {SMOOTHED_PATH}")
data_smooth = np.load(SMOOTHED_PATH, allow_pickle=True)

timestamps = data_orig["timestamps"]
vis = data_orig["aruco_visibility"]  # (N, 3) [world, object, gripper]

print(f"\nTrajectory info:")
print(f"  Num samples: {len(timestamps)}")
print(f"  Duration: {timestamps[-1]:.2f} s")


Loading original: data/trajectory_20251215_163852_7.npz
Loading smoothed: data/trajectory_20251215_163852_7_smoothed.npz

Trajectory info:
  Num samples: 402
  Duration: 20.07 s


In [39]:
# Extract original and smoothed poses
aruco_keys = [
    "aruco_ee_in_world",
    "aruco_object_in_world",
]

# Visibility masks
ee_world_vis = (vis[:, 0] > 0.5) & (vis[:, 2] > 0.5)
obj_world_vis = (vis[:, 0] > 0.5) & (vis[:, 1] > 0.5)

print("Visibility stats:")
print(f"  EE in world: {np.sum(ee_world_vis)}/{len(ee_world_vis)} frames visible")
print(f"  Object in world: {np.sum(obj_world_vis)}/{len(obj_world_vis)} frames visible")

# Helper function to draw RGB coordinate axes
def draw_rgb_axes(fig, pos, quat_wxyz, axis_length=0.02, name_prefix="", opacity=1.0):
    """Draw RGB coordinate axes (X=red, Y=green, Z=blue) at a pose."""
    # Convert quaternion [w, x, y, z] to rotation matrix
    quat_xyzw = np.array([quat_wxyz[1], quat_wxyz[2], quat_wxyz[3], quat_wxyz[0]])
    rot = R.from_quat(quat_xyzw)
    R_matrix = rot.as_matrix()
    
    # Define axis directions in local frame
    axes_local = np.array([
        [axis_length, 0, 0],  # X axis (red)
        [0, axis_length, 0],  # Y axis (green)
        [0, 0, axis_length]   # Z axis (blue)
    ])
    
    # Transform to world frame
    axes_world = (R_matrix @ axes_local.T).T + pos
    
    colors = ['red', 'green', 'blue']
    labels = ['X', 'Y', 'Z']
    
    for i in range(3):
        fig.add_trace(go.Scatter3d(
            x=[pos[0], axes_world[i, 0]],
            y=[pos[1], axes_world[i, 1]],
            z=[pos[2], axes_world[i, 2]],
            mode='lines',
            name=f'{name_prefix} {labels[i]}',
            line=dict(color=colors[i], width=4),
            showlegend=(i == 0),
            legendgroup=name_prefix,
            opacity=opacity,
        ))


Visibility stats:
  EE in world: 402/402 frames visible
  Object in world: 387/402 frames visible


In [40]:
# 3D comparison: EE trajectory with coordinate frames

ee_orig = data_orig["aruco_ee_in_world"]
ee_smooth = data_smooth["smoothed_aruco_ee_in_world"]

fig = go.Figure()

# Original trajectory (only visible frames)
fig.add_trace(go.Scatter3d(
    x=ee_orig[ee_world_vis, 0],
    y=ee_orig[ee_world_vis, 1],
    z=ee_orig[ee_world_vis, 2],
    mode="lines+markers",
    name="EE original (visible only)",
    line=dict(color="blue", width=3),
    marker=dict(size=2, color="blue"),
    opacity=0.7,
))

# Smoothed trajectory (all frames)
fig.add_trace(go.Scatter3d(
    x=ee_smooth[:, 0],
    y=ee_smooth[:, 1],
    z=ee_smooth[:, 2],
    mode="lines+markers",
    name="EE smoothed (all frames)",
    line=dict(color="cyan", width=2, dash="dash"),
    marker=dict(size=2, color="cyan"),
    opacity=0.8,
))

# Highlight interpolated frames with coordinate axes
missing_ee = ~ee_world_vis
if np.any(missing_ee):
    # Sample interpolated frames to show (every Nth frame to avoid clutter)
    missing_indices = np.where(missing_ee)[0]
    step = max(1, len(missing_indices) // 10)  # Show up to 10 frames
    sampled_missing = missing_indices[::step]
    
    # Draw coordinate frames at interpolated positions
    for idx in sampled_missing:
        pos = ee_smooth[idx, :3]
        quat = ee_smooth[idx, 3:7]
        draw_rgb_axes(fig, pos, quat, axis_length=0.01, name_prefix=f"Interp {idx}", opacity=0.6)
    
    # Also show markers
    fig.add_trace(go.Scatter3d(
        x=ee_smooth[missing_ee, 0],
        y=ee_smooth[missing_ee, 1],
        z=ee_smooth[missing_ee, 2],
        mode="markers",
        name="EE interpolated positions",
        marker=dict(size=3, color="yellow", symbol="x"),
        opacity=0.5,
    ))

# Also show frames at some visible frames for reference
if np.any(ee_world_vis):
    visible_indices = np.where(ee_world_vis)[0]
    step = max(1, len(visible_indices) // 15)  # Show up to 15 frames
    sampled_visible = visible_indices[::step]
    for idx in sampled_visible[:5]:  # Just show first few
        pos = ee_smooth[idx, :3]
        quat = ee_smooth[idx, 3:7]
        draw_rgb_axes(fig, pos, quat, axis_length=0.01, name_prefix=f"Visible {idx}", opacity=0.4)

fig.update_layout(
    title="EE Trajectory: Original vs Smoothed (with coordinate frames)",
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)",
        aspectmode="data",
    ),
    width=900,
    height=700,
    template="plotly_dark",
    paper_bgcolor="#111111",
)

fig.show()


In [41]:
# 3D comparison: Object trajectory with coordinate frames

obj_orig = data_orig["aruco_object_in_world"]
obj_smooth = data_smooth["smoothed_aruco_object_in_world"]

fig = go.Figure()

# Original trajectory (only visible frames)
fig.add_trace(go.Scatter3d(
    x=obj_orig[obj_world_vis, 0],
    y=obj_orig[obj_world_vis, 1],
    z=obj_orig[obj_world_vis, 2],
    mode="lines+markers",
    name="Object original (visible only)",
    line=dict(color="red", width=3),
    marker=dict(size=2, color="red"),
    opacity=0.7,
))

# Smoothed trajectory (all frames)
fig.add_trace(go.Scatter3d(
    x=obj_smooth[:, 0],
    y=obj_smooth[:, 1],
    z=obj_smooth[:, 2],
    mode="lines+markers",
    name="Object smoothed (all frames)",
    line=dict(color="orange", width=2, dash="dash"),
    marker=dict(size=2, color="orange"),
    opacity=0.8,
))

# Highlight interpolated frames with coordinate axes
missing_obj = ~obj_world_vis
if np.any(missing_obj):
    # Sample interpolated frames to show (every Nth frame to avoid clutter)
    missing_indices = np.where(missing_obj)[0]
    step = max(1, len(missing_indices) // 10)  # Show up to 10 frames
    sampled_missing = missing_indices[::step]
    
    # Draw coordinate frames at interpolated positions
    for idx in sampled_missing:
        pos = obj_smooth[idx, :3]
        quat = obj_smooth[idx, 3:7]
        draw_rgb_axes(fig, pos, quat, axis_length=0.01, name_prefix=f"Interp {idx}", opacity=0.6)
    
    # Also show markers
    fig.add_trace(go.Scatter3d(
        x=obj_smooth[missing_obj, 0],
        y=obj_smooth[missing_obj, 1],
        z=obj_smooth[missing_obj, 2],
        mode="markers",
        name="Object interpolated positions",
        marker=dict(size=3, color="yellow", symbol="x"),
        opacity=0.5,
    ))

# Also show frames at some visible frames for reference
if np.any(obj_world_vis):
    visible_indices = np.where(obj_world_vis)[0]
    step = max(1, len(visible_indices) // 15)  # Show up to 15 frames
    sampled_visible = visible_indices[::step]
    for idx in sampled_visible[:5]:  # Just show first few
        pos = obj_smooth[idx, :3]
        quat = obj_smooth[idx, 3:7]
        draw_rgb_axes(fig, pos, quat, axis_length=0.01, name_prefix=f"Visible {idx}", opacity=0.4)

fig.update_layout(
    title="Object Trajectory: Original vs Smoothed (with coordinate frames)",
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)",
        aspectmode="data",
    ),
    width=900,
    height=700,
    template="plotly_dark",
    paper_bgcolor="#111111",
)

fig.show()


In [42]:
# Position differences over time (EE)

pos_orig = ee_orig[:, :3]
pos_smooth = ee_smooth[:, :3]
pos_diff = np.linalg.norm(pos_orig - pos_smooth, axis=1)

fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=["Position difference (magnitude)", "X difference", "Y difference", "Z difference"],
    vertical_spacing=0.05,
)

# Magnitude
fig.add_trace(
    go.Scatter(x=timestamps, y=pos_diff, mode="lines", name="||diff||", line=dict(color="red")),
    row=1, col=1,
)

# Components
for i, label in enumerate(["X", "Y", "Z"]):
    diff_component = pos_orig[:, i] - pos_smooth[:, i]
    fig.add_trace(
        go.Scatter(x=timestamps, y=diff_component, mode="lines", name=f"{label} diff", line=dict(color=["blue", "green", "orange"][i])),
        row=i+2, col=1,
    )
    
    # Highlight missing frames
    if np.any(missing_ee):
        fig.add_trace(
            go.Scatter(
                x=timestamps[missing_ee],
                y=diff_component[missing_ee],
                mode="markers",
                name=f"{label} interpolated",
                marker=dict(size=4, color="yellow", symbol="x"),
                showlegend=(i == 0),
            ),
            row=i+2, col=1,
        )

fig.update_xaxes(title_text="Time (s)", row=4, col=1)
fig.update_yaxes(title_text="Difference (m)")

fig.update_layout(
    height=800,
    title="EE Position: Original vs Smoothed Differences",
    template="plotly_dark",
    paper_bgcolor="#111111",
    showlegend=True,
)

fig.show()

print(f"Max position difference: {pos_diff.max():.6f} m")
print(f"Mean position difference (interpolated frames only): {pos_diff[missing_ee].mean():.6f} m" if np.any(missing_ee) else "No interpolated frames")


Max position difference: 0.000000 m
No interpolated frames


In [43]:
# Quaternion differences over time (EE)

def quaternion_angle_diff(q1, q2):
    """Compute angular difference between two quaternions in radians."""
    # Convert to scipy format [x, y, z, w]
    q1_xyzw = np.array([q1[1], q1[2], q1[3], q1[0]])
    q2_xyzw = np.array([q2[1], q2[2], q2[3], q2[0]])
    
    r1 = R.from_quat(q1_xyzw)
    r2 = R.from_quat(q2_xyzw)
    
    # Relative rotation
    r_rel = r2 * r1.inv()
    return np.abs(r_rel.magnitude())

quat_orig = ee_orig[:, 3:7]
quat_smooth = ee_smooth[:, 3:7]

ang_diff = np.array([quaternion_angle_diff(quat_orig[i], quat_smooth[i]) for i in range(len(quat_orig))])

fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=["Quaternion angular difference", "Quaternion components difference"],
    vertical_spacing=0.1,
)

# Angular difference
fig.add_trace(
    go.Scatter(x=timestamps, y=np.degrees(ang_diff), mode="lines", name="Angle diff (deg)", line=dict(color="purple")),
    row=1, col=1,
)

# Highlight missing frames
if np.any(missing_ee):
    fig.add_trace(
        go.Scatter(
            x=timestamps[missing_ee],
            y=np.degrees(ang_diff[missing_ee]),
            mode="markers",
            name="Interpolated",
            marker=dict(size=4, color="yellow", symbol="x"),
        ),
        row=1, col=1,
    )

# Component differences
quat_diff = quat_orig - quat_smooth
labels = ["qw", "qx", "qy", "qz"]
colors = ["red", "blue", "green", "orange"]
for i, (label, color) in enumerate(zip(labels, colors)):
    fig.add_trace(
        go.Scatter(x=timestamps, y=quat_diff[:, i], mode="lines", name=f"{label} diff", line=dict(color=color)),
        row=2, col=1,
    )

fig.update_xaxes(title_text="Time (s)", row=2, col=1)
fig.update_yaxes(title_text="Angle (deg)", row=1, col=1)
fig.update_yaxes(title_text="Quaternion component diff", row=2, col=1)

fig.update_layout(
    height=600,
    title="EE Orientation: Original vs Smoothed Differences",
    template="plotly_dark",
    paper_bgcolor="#111111",
    showlegend=True,
)

fig.show()

print(f"Max angular difference: {np.degrees(ang_diff.max()):.2f} deg")
print(f"Mean angular difference (interpolated frames only): {np.degrees(ang_diff[missing_ee].mean()):.2f} deg" if np.any(missing_ee) else "No interpolated frames")


Max angular difference: 0.00 deg
No interpolated frames


In [44]:
# Summary: Before/After comparison side-by-side

print("="*60)
print("SMOOTHING SUMMARY")
print("="*60)
print(f"\nEE in World:")
print(f"  Original visible frames: {np.sum(ee_world_vis)}/{len(ee_world_vis)}")
print(f"  Interpolated frames: {np.sum(missing_ee)}")
if np.any(missing_ee):
    print(f"  Max position diff: {pos_diff[missing_ee].max():.6f} m")
    print(f"  Mean position diff: {pos_diff[missing_ee].mean():.6f} m")
    print(f"  Max angular diff: {np.degrees(ang_diff[missing_ee].max()):.2f} deg")
    print(f"  Mean angular diff: {np.degrees(ang_diff[missing_ee].mean()):.2f} deg")

print(f"\nObject in World:")
print(f"  Original visible frames: {np.sum(obj_world_vis)}/{len(obj_world_vis)}")
print(f"  Interpolated frames: {np.sum(~obj_world_vis)}")

print("\nNote: Differences are only non-zero for interpolated frames.")
print("Original visible frames remain unchanged in smoothed version.")


SMOOTHING SUMMARY

EE in World:
  Original visible frames: 402/402
  Interpolated frames: 0

Object in World:
  Original visible frames: 387/402
  Interpolated frames: 15

Note: Differences are only non-zero for interpolated frames.
Original visible frames remain unchanged in smoothed version.


In [45]:
# Visualize coordinate frames to see rotation interpolation
# Draw RGB axes (X=red, Y=green, Z=blue) at key timesteps

def draw_rgb_axes(fig, pos, quat_wxyz, axis_length=0.02, name_prefix="", opacity=1.0):
    """Draw RGB coordinate axes at a pose."""
    # Convert quaternion [w, x, y, z] to rotation matrix
    quat_xyzw = np.array([quat_wxyz[1], quat_wxyz[2], quat_wxyz[3], quat_wxyz[0]])
    rot = R.from_quat(quat_xyzw)
    R_matrix = rot.as_matrix()
    
    # Define axis directions in local frame
    axes_local = np.array([
        [axis_length, 0, 0],  # X axis (red)
        [0, axis_length, 0],  # Y axis (green)
        [0, 0, axis_length]   # Z axis (blue)
    ])
    
    # Transform to world frame
    axes_world = (R_matrix @ axes_local.T).T + pos
    
    colors = ['red', 'green', 'blue']
    labels = ['X', 'Y', 'Z']
    
    for i in range(3):
        fig.add_trace(go.Scatter3d(
            x=[pos[0], axes_world[i, 0]],
            y=[pos[1], axes_world[i, 1]],
            z=[pos[2], axes_world[i, 2]],
            mode='lines',
            name=f'{name_prefix} {labels[i]}',
            line=dict(color=colors[i], width=4),
            showlegend=(i == 0),
            legendgroup=name_prefix,
            opacity=opacity,
        ))

# Find an interpolated region to visualize
missing_obj = ~obj_world_vis
if np.any(missing_obj):
    # Find first contiguous missing region
    missing_indices = np.where(missing_obj)[0]
    if len(missing_indices) > 0:
        # Get a region around the first missing frame
        start_missing = missing_indices[0]
        # Find the end of this contiguous region
        end_missing = start_missing
        while end_missing + 1 < len(missing_obj) and missing_obj[end_missing + 1]:
            end_missing += 1
        
        # Show frames before, during, and after interpolation
        before_idx = max(0, start_missing - 1)
        after_idx = min(len(obj_orig) - 1, end_missing + 1)
        
        # Sample frames: before, during (every 2nd frame), and after
        frame_indices = [before_idx]
        if end_missing > start_missing:
            frame_indices.extend(range(start_missing, end_missing + 1, max(1, (end_missing - start_missing) // 3)))
        else:
            frame_indices.append(start_missing)
        frame_indices.append(after_idx)
        frame_indices = sorted(set(frame_indices))
        
        print(f"Visualizing rotation interpolation around frames {start_missing}-{end_missing}")
        print(f"Showing frames: {frame_indices}")
        
        fig = go.Figure()
        
        # Draw trajectory path
        fig.add_trace(go.Scatter3d(
            x=obj_smooth[:, 0],
            y=obj_smooth[:, 1],
            z=obj_smooth[:, 2],
            mode='lines',
            name='Object path',
            line=dict(color='gray', width=2),
            opacity=0.3,
        ))
        
        # Draw frames
        for idx in frame_indices:
            pos = obj_smooth[idx, :3]
            quat = obj_smooth[idx, 3:7]
            
            if idx < start_missing:
                prefix = f"Before ({idx})"
                opacity = 1.0
            elif idx > end_missing:
                prefix = f"After ({idx})"
                opacity = 1.0
            else:
                prefix = f"Interp ({idx})"
                opacity = 0.7
            
            draw_rgb_axes(fig, pos, quat, axis_length=0.015, name_prefix=prefix, opacity=opacity)
        
        fig.update_layout(
            title=f"Rotation Interpolation: Object Frames Around Missing Region (frames {start_missing}-{end_missing})",
            scene=dict(
                xaxis_title='X (m)',
                yaxis_title='Y (m)',
                zaxis_title='Z (m)',
                aspectmode='data',
            ),
            width=1000,
            height=800,
            template="plotly_dark",
            paper_bgcolor="#111111",
        )
        
        fig.show()
    else:
        print("No missing frames found for object")
else:
    print("No interpolated frames in object trajectory")


Visualizing rotation interpolation around frames 189-189
Showing frames: [np.int64(188), np.int64(189), np.int64(190)]


In [46]:
# Visualize EE frames if there are interpolated regions
missing_ee = ~ee_world_vis
if np.any(missing_ee):
    missing_indices = np.where(missing_ee)[0]
    if len(missing_indices) > 0:
        start_missing = missing_indices[0]
        end_missing = start_missing
        while end_missing + 1 < len(missing_ee) and missing_ee[end_missing + 1]:
            end_missing += 1
        
        before_idx = max(0, start_missing - 1)
        after_idx = min(len(ee_orig) - 1, end_missing + 1)
        
        frame_indices = [before_idx]
        if end_missing > start_missing:
            frame_indices.extend(range(start_missing, end_missing + 1, max(1, (end_missing - start_missing) // 3)))
        else:
            frame_indices.append(start_missing)
        frame_indices.append(after_idx)
        frame_indices = sorted(set(frame_indices))
        
        print(f"Visualizing EE rotation interpolation around frames {start_missing}-{end_missing}")
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter3d(
            x=ee_smooth[:, 0],
            y=ee_smooth[:, 1],
            z=ee_smooth[:, 2],
            mode='lines',
            name='EE path',
            line=dict(color='gray', width=2),
            opacity=0.3,
        ))
        
        for idx in frame_indices:
            pos = ee_smooth[idx, :3]
            quat = ee_smooth[idx, 3:7]
            
            if idx < start_missing:
                prefix = f"Before ({idx})"
                opacity = 1.0
            elif idx > end_missing:
                prefix = f"After ({idx})"
                opacity = 1.0
            else:
                prefix = f"Interp ({idx})"
                opacity = 0.7
            
            draw_rgb_axes(fig, pos, quat, axis_length=0.015, name_prefix=prefix, opacity=opacity)
        
        fig.update_layout(
            title=f"Rotation Interpolation: EE Frames Around Missing Region (frames {start_missing}-{end_missing})",
            scene=dict(
                xaxis_title='X (m)',
                yaxis_title='Y (m)',
                zaxis_title='Z (m)',
                aspectmode='data',
            ),
            width=1000,
            height=800,
            template="plotly_dark",
            paper_bgcolor="#111111",
        )
        
        fig.show()
    else:
        print("No missing frames found for EE")
else:
    print("No interpolated frames in EE trajectory - all frames were visible")


No interpolated frames in EE trajectory - all frames were visible


In [47]:
# Side-by-side comparison: Original vs Smoothed frames at interpolated region

missing_obj = ~obj_world_vis
if np.any(missing_obj):
    missing_indices = np.where(missing_obj)[0]
    start_missing = missing_indices[0]
    end_missing = start_missing
    while end_missing + 1 < len(missing_obj) and missing_obj[end_missing + 1]:
        end_missing += 1
    
    before_idx = max(0, start_missing - 1)
    after_idx = min(len(obj_orig) - 1, end_missing + 1)
    
    # Show key frames: before, middle of gap, after
    key_frames = [before_idx]
    if end_missing > start_missing:
        mid_idx = (start_missing + end_missing) // 2
        key_frames.extend([start_missing, mid_idx, end_missing])
    else:
        key_frames.append(start_missing)
    key_frames.append(after_idx)
    key_frames = sorted(set(key_frames))
    
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        subplot_titles=('Original (visible frames only)', 'Smoothed (with interpolation)'),
    )
    
    # Original side
    fig.add_trace(go.Scatter3d(
        x=obj_orig[obj_world_vis, 0],
        y=obj_orig[obj_world_vis, 1],
        z=obj_orig[obj_world_vis, 2],
        mode='lines',
        name='Original path',
        line=dict(color='gray', width=2),
        opacity=0.3,
        showlegend=False,
    ), row=1, col=1)
    
    # Draw original frames (only visible ones)
    for idx in key_frames:
        if obj_world_vis[idx]:
            pos = obj_orig[idx, :3]
            quat = obj_orig[idx, 3:7]
            if idx < start_missing:
                prefix = f"Before ({idx})"
            elif idx > end_missing:
                prefix = f"After ({idx})"
            else:
                continue  # Skip missing frames in original
            
            # Draw axes manually for subplot
            quat_xyzw = np.array([quat[1], quat[2], quat[3], quat[0]])
            rot = R.from_quat(quat_xyzw)
            R_matrix = rot.as_matrix()
            axis_length = 0.015
            axes_local = np.array([[axis_length, 0, 0], [0, axis_length, 0], [0, 0, axis_length]])
            axes_world = (R_matrix @ axes_local.T).T + pos
            colors = ['red', 'green', 'blue']
            
            for i in range(3):
                fig.add_trace(go.Scatter3d(
                    x=[pos[0], axes_world[i, 0]],
                    y=[pos[1], axes_world[i, 1]],
                    z=[pos[2], axes_world[i, 2]],
                    mode='lines',
                    name=f'{prefix} {["X","Y","Z"][i]}',
                    line=dict(color=colors[i], width=3),
                    showlegend=bool(idx == before_idx and i == 0),
                    legendgroup=f'orig_{prefix}',
                ), row=1, col=1)
    
    # Smoothed side
    fig.add_trace(go.Scatter3d(
        x=obj_smooth[:, 0],
        y=obj_smooth[:, 1],
        z=obj_smooth[:, 2],
        mode='lines',
        name='Smoothed path',
        line=dict(color='gray', width=2),
        opacity=0.3,
        showlegend=False,
    ), row=1, col=2)
    
    # Draw smoothed frames (all key frames)
    for idx in key_frames:
        pos = obj_smooth[idx, :3]
        quat = obj_smooth[idx, 3:7]
        
        if idx < start_missing:
            prefix = f"Before ({idx})"
            opacity = 1.0
        elif idx > end_missing:
            prefix = f"After ({idx})"
            opacity = 1.0
        else:
            prefix = f"Interp ({idx})"
            opacity = 0.7
        
        quat_xyzw = np.array([quat[1], quat[2], quat[3], quat[0]])
        rot = R.from_quat(quat_xyzw)
        R_matrix = rot.as_matrix()
        axis_length = 0.015
        axes_local = np.array([[axis_length, 0, 0], [0, axis_length, 0], [0, 0, axis_length]])
        axes_world = (R_matrix @ axes_local.T).T + pos
        
        for i in range(3):
            fig.add_trace(go.Scatter3d(
                x=[pos[0], axes_world[i, 0]],
                y=[pos[1], axes_world[i, 1]],
                z=[pos[2], axes_world[i, 2]],
                mode='lines',
                name=f'{prefix} {["X","Y","Z"][i]}',
                line=dict(color=colors[i], width=3),
                opacity=opacity,
                showlegend=bool(idx == key_frames[0] and i == 0),
                legendgroup=f'smooth_{prefix}',
            ), row=1, col=2)
    
    fig.update_layout(
        title=f"Frame Comparison: Original vs Smoothed (around frames {start_missing}-{end_missing})",
        width=1800,
        height=800,
        template="plotly_dark",
        paper_bgcolor="#111111",
    )
    
    # Update scene for both subplots
    for col in [1, 2]:
        fig.update_scenes(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data',
            row=1, col=col,
        )
    
    fig.show()
